In [1]:
from script.scraper_functions import *
from script.soupifier import *
from serde.json import to_json
import numpy as np
from itertools import chain

## Le code ci-dessous lance le `scraper`

- Tu peux changer le nombre d'*URL* de **19** à 1 ou 2 pour tester ça prendra beaucoup moins longtemps à générer
- Lien [`Darty`](https://www.darty.com/nav/extra/list?seller=311289-0-327183-311290-311291-328771-311452&cat=89054&state=new_offer&persist_sold_by_darty=true) $\Rightarrow$ en choisissant les produits uniquement vendus par Darty + Neufs on a 567 produits.
- Avoir un comparatif Boulanger/Darty pourrait être intéressant
- Voir pour récupérer les images des téléphones pour plus tard dans l'appli - voir : https://discuss.streamlit.io/t/add-image-and-header-to-streamlit-dataframe-table/36065/3

In [2]:
session = create_session()
url_list = create_url_list(URL, 1)
pages = read_pages(url_list=url_list, session=session)

Page 1 -> Successfull Extraction.


Elapsed time for read_pages function: 8.326 seconds.

In [3]:
all_urls = extract_all_urls(pages)
print("--------------------")
product_pages = extract_all_pages(all_urls, session)

Elapsed time for extract_all_urls function: 0.488 seconds.

--------------------
Product 1 -> Successfull Extraction.
Product 2 -> Successfull Extraction.
Product 3 -> Successfull Extraction.
Product 4 -> Successfull Extraction.
Product 5 -> Successfull Extraction.
Product 6 -> Successfull Extraction.
Product 7 -> Successfull Extraction.
Product 8 -> Successfull Extraction.
Product 9 -> Successfull Extraction.
Product 10 -> Successfull Extraction.
Product 11 -> Successfull Extraction.
Product 12 -> Successfull Extraction.
Product 13 -> Successfull Extraction.
Product 14 -> Successfull Extraction.
Product 15 -> Successfull Extraction.
Product 16 -> Successfull Extraction.
Product 17 -> Successfull Extraction.
Product 18 -> Successfull Extraction.
Product 19 -> Successfull Extraction.
Product 20 -> Successfull Extraction.
Product 21 -> Successfull Extraction.
Product 22 -> Successfull Extraction.
Product 23 -> Successfull Extraction.
Product 24 -> Successfull Extraction.
Product 25 -> Successfull Extraction.
Product 26 -> Successfull Extraction.


Elapsed time for extract_all_pages function: 59.688 seconds.

Il faudrait créer une fonction pour la génération de la liste des smartphones + l'export en `json`

In [8]:
smartphone_list = list()
for article in product_pages:
    soup = BeautifulSoup(article, "html.parser")
    dict_features = extract_features(soup)
    smartphone = smartphone_characteristics(dict_features)
    smartphone_list.append(smartphone)

In [13]:
smartphones_json = to_json(smartphone_list)

In [14]:
file_path = "data/smartphones.json"

with open(file_path, "w", encoding="utf-8") as json_file:
    json_file.write(smartphones_json)

## Charge le Dataframe dans `polars`

In [1]:
import polars as pl
from script.polars_gazprom import *

In [2]:
df_brut = pl.read_json("data/smartphones.json")

### Pipeline

Sur `535` extractions, il y a `436` téléphones valides avec les caractéristiques. Pas mal ?

Pour obtenir la *PPI* (Pixels per Inch) on fait : 

$$PPI = \dfrac{\text{diagonal\_pixels}}{\text{screen\_size} (inch)}$$

In [4]:
df = NordStream(df_brut)

In [5]:
df

sensor,width,screen_type,charging_type,sensor_resolution,made_in,usb_type_c,thickness,stars,battery,ram,das_head,upgrade_storage,das_chest,price,reviews,fast_charging,cpu,brand,storage,net_weight,height,screen_size,network,color,das_limbs,sim_type,model,screen_tech,repairability_index,diagonal_pixels
str,str,str,str,str,str,bool,str,f64,i64,str,str,bool,str,f64,i64,bool,str,str,str,str,str,f64,str,str,str,str,str,str,str,f64
"""3""","""0,70 mm""","""Plat""","""Possibilité de…","""12 mégapixels+…","""Thaïlande""",true,"""0,90 mm""",0.0,5000,"""8 Go""","""0,950 W/kg""",true,"""1,050 W/kg""",999.0,0,true,"""Qualcomm Snapd…","""SONY""","""128 Go""","""0,00 g""","""15,40 mm""",6.1,"""5G""","""Argent""","""2,110 W/kg""","""Nano SIM et eS…","""SONY Xperia 5 …","""OLED FHD+""","""8,5 /10""",2741.678318
"""3""","""87,00 mm""","""Plat""","""Possibilité de…","""50 mégapixels+…","""Viêt Nam""",true,"""0,80 mm""",4.0,3700,"""8 Go""","""1,210 W/kg""",false,"""1,590 W/kg""",857.59,1,true,"""Exynos 2200""","""SAMSUNG""","""256 Go""","""167,00 g""","""146,00 mm""",6.1,"""5G""","""Noir""","""2,710 W/kg""","""Nano SIM et eS…","""Samsung Galaxy…","""Dynamic AMOLED…","""8,2 /10""",2577.207791
"""2""","""74,70 mm""","""Plat""","""Pas de charge …","""50 mégapixels+…","""Chine""",true,"""8,20 mm""",4.3333,5000,"""4 Go""","""0,980 W/kg""",true,"""1,320 W/kg""",199.0,3,false,"""8 coeurs jusqu…","""MOTOROLA""","""128 Go""","""183,00 g""","""162,70 mm""",6.5,"""5G""","""Bleu""","""2,980 W/kg""","""Nano SIM et eS…","""Motorola G53""","""LCD""","""7,1 /10""",1754.536976
"""3""","""7,10 mm""","""Plat""","""Possibilité de…","""50 mégapixels+…","""Viêt Nam""",true,"""0,90 mm""",0.0,4500,"""8 Go""","""0,980 W/kg""",false,"""1,200 W/kg""",799.0,0,true,"""Google Tensor …","""GOOGLE""","""128 Go""",null,"""15,10 mm""",6.2,"""5G""","""Noir""","""2,490 W/kg""","""2Nano sim ou 1…","""Google Pixel 8…","""Actua ou OLED …","""8,2 /10""",2631.805464
"""1""","""77,10 mm""","""Plat""","""Pas de charge …","""13 mégapixels""","""Chine""",true,"""9,00 mm""",4.0,5000,"""2 Go""","""0,398 W/kg""",true,"""1,186 W/kg""",95.9,2,true,"""8 coeurs jusqu…","""XIAOMI""","""32 Go""","""194,00 g""","""164,90 mm""",6.5,"""4G+""","""Gris""","""2,526 W/kg""","""Nano SIM (x2) …","""Xiaomi Redmi 9…","""HD+""","""7,0 /10""",1754.536976
"""3""","""73,90 mm""","""Bords incurvés…","""Pas de charge …","""64 mégapixels+…","""Chine""",true,"""7,90 mm""",4.1333,5100,"""8 Go""","""0,820 W/kg""",false,"""1,270 W/kg""",299.0,15,true,"""Qualcomm Snapd…","""HONOR""","""256 Go""","""175,00 g""","""161,60 mm""",6.7,"""5G""","""Noir""","""2,810 W/kg""","""Nano SIM (x2)""","""HONOR Magic5 L…","""AMOLED (rigide…","""8,0 /10""",2631.805464
"""3""","""75,90 mm""","""Plat""","""Possibilité de…","""50 mégapixels+…","""Chine""",true,"""8,90 mm""",4.087,5000,"""12 Go""","""0,990 W/kg""",false,"""1,400 W/kg""",633.67,23,true,"""Google Tensor""","""GOOGLE""","""128 Go""","""210,00 g""","""163,90 mm""",6.7,"""5G""","""Noir""","""2,960 W/kg""","""Dual SIM""","""Google Pixel 6…","""Non communiqué…","""6,4 /10""",3436.277055
null,"""73,50 mm""","""Borderless""","""Pas de charge …","""Non communiqué…","""Chine""",true,"""8,00 mm""",0.0,4300,"""6 Go""","""1,139 W/kg""",true,"""1,180 W/kg""",304.25,0,true,"""Non communiqué…","""REALME""","""128 Go""","""174,00 g""","""159,10 mm""",6.4,"""5G""","""Noir""","""1,880 W/kg""","""Nano SIM""","""Realme GT Mast…","""Non communiqué…","""6,2 /10""",2631.805464
"""3""","""7,10 mm""","""Plat""","""Possibilité de…","""50 mégapixels+…","""Viêt Nam""",true,"""0,90 mm""",0.0,4500,"""8 Go""","""0,980 W/kg""",false,"""1,200 W/kg""",859.0,0,true,"""Google Tensor …","""GOOGLE""","""256 Go""",null,"""15,10 mm""",6.2,"""5G""","""Vert""","""2,490 W/kg""","""2Nano sim ou 1…","""Google Pixel 8…","""Actua ou OLED …","""8,2 /10""",2631.805464


In [26]:
df.select("model", "stars", "reviews").filter(
    (pl.col("reviews") > 0) & (pl.col("stars") < 4)
).to_numpy()

array([['Realme 9 Pro', 3.6666998863220215, 3],
       ['ASUS ROG Phone 6', 3.0, 1],
       ['Google Pixel 7a', 2.75, 4],
       ['XIAOMI Redmi Note 12', 3.4000000953674316, 5],
       ['OPPO Find X5 Lite', 3.5999999046325684, 15],
       ['Honor X7A', 3.923099994659424, 13],
       ['Xiaomi 13', 3.0, 1],
       ['ASUS ROG Phone 6', 3.0, 1],
       ['Samsung Galaxy A13', 3.6666998863220215, 3],
       ['Sony Xperia 5 IV', 3.4000000953674316, 5],
       ['XIAOMI Redmi Note 12 Pro', 3.3076999187469482, 13],
       ['Xiaomi Redmi 9A', 3.0, 1],
       ['Xiaomi 11T', 3.9544999599456787, 22],
       ['iPhone 14', 1.0, 1],
       ['XIAOMI Redmi 10A', 3.5, 2],
       ['SONY Xperia 10 V', 3.0, 3],
       ['Xiaomi Redmi 12 4G', 3.5999999046325684, 10],
       ['VIVO V21', 3.727299928665161, 11],
       ['Google Pixel 7a', 3.0, 1],
       ['Xiaomi Redmi Note 11', 3.766700029373169, 30],
       ['Xiaomi Redmi A1', 2.0, 1],
       ['SONY Xperia 10 V', 3.0, 1],
       ['Xiaomi Redmi A1', 3.799999952

In [5]:
df.group_by("brand").count()

brand,count
str,u32
"""APPLE""",46
"""REALME""",14
"""SAMSUNG""",129
"""HONOR""",14
"""NOTHING""",7
"""XIAOMI""",105
"""SONY""",15
"""VIVO""",9
"""FAIRPHONE""",3


- l'OS est il à garder ? pas sûr car il n'y a en vérité que deux choix d'OS aujourd'hui, et étant donné que l'information est forcément comprise dans les marques (Apple = iOS) et (Tous les autres = Android) ce n'est pas forcément utile

In [58]:
df.select("price", "storage").group_by("storage").mean().sort("price")

storage,price
str,f64
"""32 Go""",116.647407
"""64 Go""",211.716964
"""128 Go""",495.199235
"""256 Go""",884.259464
"""512 Go""",1214.744324
"""1000 Go""",1752.29375


In [59]:
df.select("price", "ram").group_by("ram").mean().sort("price")

ram,price
str,f64
"""2 Go""",110.109048
"""3 Go""",158.240588
"""4 Go""",413.402542
"""6 Go""",462.522203
"""8 Go""",686.67741
"""16 Go""",1106.142857
"""12 Go""",1144.023733


In [34]:
df

os,stars,model,screen_type,repairability_index,brand,made_in,das_limbs,network,width,height,reviews,screen_size,price,fast_charging,net_weight,storage,usb_type_c,ram,das_chest,battery,upgrade_storage,das_head,sensor_resolution,cpu,color,charging_type,screen_tech,sim_type,sensor,thickness,diagonal_pixels
str,f64,str,str,str,str,str,str,str,str,str,i64,f64,f64,bool,str,str,bool,str,str,i64,bool,str,str,str,str,str,str,str,str,str,f64
"""Android 13""",0.0,"""SONY Xperia 5 …","""Plat""","""8,5 /10""","""SONY""","""Thaïlande""","""2,110 W/kg""","""5G""","""0,70 mm""","""15,40 mm""",0,6.1,999.0,true,"""0,00 g""","""128 Go""",true,"""8 Go""","""1,050 W/kg""",5000,true,"""0,950 W/kg""","""12 mégapixels+…","""Qualcomm Snapd…","""Argent""","""Possibilité de…","""OLED FHD+""","""Nano SIM et eS…","""3""","""0,90 mm""",2741.678318
"""Android 12""",4.0,"""Samsung Galaxy…","""Plat""","""8,2 /10""","""SAMSUNG""","""Viêt Nam""","""2,710 W/kg""","""5G""","""87,00 mm""","""146,00 mm""",1,6.1,857.59,true,"""167,00 g""","""256 Go""",true,"""8 Go""","""1,590 W/kg""",3700,false,"""1,210 W/kg""","""50 mégapixels+…","""Exynos 2200""","""Noir""","""Possibilité de…","""Dynamic AMOLED…","""Nano SIM et eS…","""3""","""0,80 mm""",2577.207791
"""Android 13""",4.3333,"""Motorola G53""","""Plat""","""7,1 /10""","""MOTOROLA""","""Chine""","""2,980 W/kg""","""5G""","""74,70 mm""","""162,70 mm""",3,6.5,199.0,false,"""183,00 g""","""128 Go""",true,"""4 Go""","""1,320 W/kg""",5000,true,"""0,980 W/kg""","""50 mégapixels+…","""8 coeurs jusqu…","""Bleu""","""Pas de charge …","""LCD""","""Nano SIM et eS…","""2""","""8,20 mm""",1754.536976
"""Android 13""",0.0,"""Google Pixel 8…","""Plat""","""8,2 /10""","""GOOGLE""","""Viêt Nam""","""2,490 W/kg""","""5G""","""7,10 mm""","""15,10 mm""",0,6.2,799.0,true,null,"""128 Go""",true,"""8 Go""","""1,200 W/kg""",4500,false,"""0,980 W/kg""","""50 mégapixels+…","""Google Tensor …","""Noir""","""Possibilité de…","""Actua ou OLED …","""2Nano sim ou 1…","""3""","""0,90 mm""",2631.805464
"""Android 11""",4.0,"""Xiaomi Redmi 9…","""Plat""","""7,0 /10""","""XIAOMI""","""Chine""","""2,526 W/kg""","""4G+""","""77,10 mm""","""164,90 mm""",2,6.5,95.9,true,"""194,00 g""","""32 Go""",true,"""2 Go""","""1,186 W/kg""",5000,true,"""0,398 W/kg""","""13 mégapixels""","""8 coeurs jusqu…","""Gris""","""Pas de charge …","""HD+""","""Nano SIM (x2) …","""1""","""9,00 mm""",1754.536976
"""Android S + Ma…",4.1333,"""HONOR Magic5 L…","""Bords incurvés…","""8,0 /10""","""HONOR""","""Chine""","""2,810 W/kg""","""5G""","""73,90 mm""","""161,60 mm""",15,6.7,299.0,true,"""175,00 g""","""256 Go""",true,"""8 Go""","""1,270 W/kg""",5100,false,"""0,820 W/kg""","""64 mégapixels+…","""Qualcomm Snapd…","""Noir""","""Pas de charge …","""AMOLED (rigide…","""Nano SIM (x2)""","""3""","""7,90 mm""",2631.805464
"""Android 12""",4.087,"""Google Pixel 6…","""Plat""","""6,4 /10""","""GOOGLE""","""Chine""","""2,960 W/kg""","""5G""","""75,90 mm""","""163,90 mm""",23,6.7,633.67,true,"""210,00 g""","""128 Go""",true,"""12 Go""","""1,400 W/kg""",5000,false,"""0,990 W/kg""","""50 mégapixels+…","""Google Tensor""","""Noir""","""Possibilité de…","""Non communiqué…","""Dual SIM""","""3""","""8,90 mm""",3436.277055
"""Non communiqué…",0.0,"""Realme GT Mast…","""Borderless""","""6,2 /10""","""REALME""","""Chine""","""1,880 W/kg""","""5G""","""73,50 mm""","""159,10 mm""",0,6.4,304.25,true,"""174,00 g""","""128 Go""",true,"""6 Go""","""1,180 W/kg""",4300,true,"""1,139 W/kg""","""Non communiqué…","""Non communiqué…","""Noir""","""Pas de charge …","""Non communiqué…","""Nano SIM""",null,"""8,00 mm""",2631.805464
"""Android 13""",0.0,"""Google Pixel 8…","""Plat""","""8,2 /10""","""GOOGLE""","""Viêt Nam""","""2,490 W/kg""","""5G""","""7,10 mm""","""15,10 mm""",0,6.2,859.0,true,null,"""256 Go""",true,"""8 Go""","""1,200 W/kg""",4500,false,"""0,980 W/kg""","""50 mégapixels+…","""Google Tensor …","""Vert""","""Possibilité de…","""Actua ou OLED …","""2Nano sim ou 1…","""3""","""0,90 mm""",2631.80

In [17]:
import plotly.express as px

fig = px.histogram(df, x="diagonal_pixels", histnorm="probability")
fig.update_layout(
    title="Distribution des diagonal_pixels",
    xaxis_title="diagonal_pixels",
    yaxis_title="Fréquence",
)
fig.show()

**TODO** `cleaning` :

- [x] **network**
- [x] **color**
- [x] **price**
- [x] **screen_type**
- [ ] *cpu*

In [10]:
# df.select("cpu").unique().to_numpy()
# pour le cpu on va préférer retravailler pour avoir la fréquence en GHz plutot

Vérification des valeurs nulles sur les différentes colonnes.

In [5]:
df.select(pl.all().is_null().sum())

fast_charging,reviews,screen_size,das_chest,network,stars,screen_type,sensor_resolution,os,color,das_limbs,net_weight,storage,ram,usb_type_c,price,battery,das_head,cpu,made_in,width,model,charging_type,screen_tech,upgrade_storage,height,repairability_index,thickness,sim_type,brand,sensor,total_pixels
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,1,1,0,14,0,0,0,0,0,0,1,7,1,0,3,0,0,1,0,1,0,0,75,0


In [5]:
df.select(pl.col("model").filter(pl.col("net_weight").is_null())).unique().to_numpy()

array([['Samsung Galaxy S10'],
       ['Google Pixel 8'],
       ['Google Pixel 8 Pro'],
       ['Sony Xperia 1 V']], dtype=object)

Idée => Avec cette liste unique des téléphones, aller chercher leur poids (manquant) avec une `request` sur google et récupérer le résultat

In [8]:
df.filter(pl.col("charging_type").is_null())
# ces 3 téléphones sont en fait chargeables par induction
# renommer la colonne charging_type par induction et avoir des valeurs true/false booléennes

stars,thickness,screen_tech,screen_resolution,net_weight,das_head,charging_type,os,brand,repairability_index,bluetooth,sim_number,network,width,sim_type,color,screen_size,sensor,screen_type,fast_charging,usb_type_c,sensor_resolution,reviews,storage,battery,height,model,ram,das_limbs,cpu,made_in,das_chest,price,upgrade_storage
f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,bool,str,i64,str,str,str,str,str,str,str,str,str,f64,bool
4.3137,"""8,40 mm""","""TFT (LCD)""","""2408 x 1080 pi…","""197,00 g""","""1,492 W/kg""",null,"""Android 12""","""SAMSUNG""","""8,1 /10""","""5.1""","""2 SIM""","""5G""","""76,90 mm""","""Dual SIM""","""Bleu""","""6,6"" soit 16,8…","""3""","""Plat""","""Oui""",true,"""50 mégapixels+…",51,"""128 Go""","""5000 mAh""","""165,40 mm""","""Samsung Galaxy…","""4 Go""","""2,840 W/kg""","""8 coeurs jusqu…","""Viêt Nam""","""1,251 W/kg""",279.0,true
4.3137,"""8,40 mm""","""TFT (LCD)""","""2408 x 1080 pi…","""197,00 g""","""1,492 W/kg""",null,"""Android 12""","""SAMSUNG""","""8,1 /10""","""5.1""","""2 SIM""","""5G""","""76,90 mm""","""Dual SIM""","""Noir""","""6,6"" soit 16,8…","""3""","""Plat""","""Oui""",true,"""50 mégapixels+…",51,"""128 Go""","""5000 mAh""","""165,40 mm""","""Samsung Galaxy…","""4 Go""","""2,840 W/kg""","""8 coeurs jusqu…","""Viêt Nam""","""1,251 W/kg""",279.0,true
4.6667,"""8,40 mm""","""LCD (LTPS) FHD…","""1080 x 2400 pi…","""190,00 g""","""0,788 W/kg""",null,"""Android 11""","""REALME""","""6,1 /10""","""5""","""1 SIM""","""4G+""","""75,50 mm""","""Nano SIM""","""Noir""","""6,6"" soit 16,8…","""3""","""Plat""","""Oui""",true,"""50 mégapixels+…",3,"""64 Go""","""5000 mAh""","""164,40 mm""","""Realme 9i""","""4 Go""","""2,072 W/kg""","""8 coeurs jusqu…","""Chine""","""1,020 W/kg""",219.0,true


In [13]:
df.group_by("color").count().sort("count", descending=True).to_numpy()

array([['Noir', 167],
       ['Bleu', 68],
       ['Gris', 54],
       ['Vert', 50],
       ['Blanc', 40],
       ['Violet', 23],
       ['Beige', 10],
       ['Argent', 9],
       ['Rose', 5],
       ['Rouge', 5],
       ['Jaune', 3],
       ['Bronze', 2],
       ['Orange', 1],
       [None, 1],
       ['Or', 1]], dtype=object)